In [ ]:
!pip install timm flash-attn transformers datasets bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have p

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor, get_scheduler
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

from bitsandbytes.optim import AdamW
from tqdm import tqdm

In [ ]:
model_id = 'microsoft/Florence-2-large'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map='cuda')
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("derek-thomas/ScienceQA")

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

In [ ]:
dataset.set_format("pandas")

In [ ]:
dataset['train'][:3]

,image,question,choices,answer,hint,task,grade,subject,topic,category,skill,lecture,solution
0,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Which of these states is farthest north?,"[West Virginia, Louisiana, Arizona, Oklahoma]",0,,closed choice,grade2,social science,geography,Geography,Read a map: cardinal directions,"Maps have four cardinal directions, or main di...","To find the answer, look at the compass rose. ..."
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Identify the question that Tom and Justin's ex...,[Do ping pong balls stop rolling along the gro...,1,The passage below describes an experiment. Rea...,closed choice,grade8,natural science,science-and-engineering-practices,Designing experiments,Identify the experimental question,Experiments can be designed to answer specific...,
2,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Identify the question that Kathleen and Bryant...,[Does Kathleen's snowboard slide down a hill i...,0,The passage below describes an experiment. Rea...,closed choice,grade7,natural science,science-and-engineering-practices,Designing experiments,Identify the experimental question,Experiments can be designed to answer specific...,


In [ ]:
for name, child in model.named_children():
    print(f"Submodule name: {name}, Type: {type(child).__name__}")

Submodule name: vision_tower, Type: DaViT
Submodule name: image_proj_norm, Type: LayerNorm
Submodule name: image_pos_embed, Type: LearnedAbsolutePositionEmbedding2D
Submodule name: visual_temporal_embed, Type: PositionalEmbeddingCosine1D
Submodule name: language_model, Type: Florence2LanguageForConditionalGeneration


In [ ]:
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": True})
for params in model.vision_tower.parameters():
  params.is_trainable=False

In [ ]:
dataset.reset_format()

In [ ]:
class MMInstructDataset(Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    temp = self.data[idx]
    question = "\n"+temp['question']+"\n\n"
    choice = temp['choices']
    answer = choice[temp['answer']]
    if temp['image'] is not None:
      image = temp['image'].convert("RGB")
    else:
      dummy = np.random.rand(50, 50)
      image = Image.fromarray(dummy).convert("RGB")
    return question, answer, image

In [ ]:
train_data = MMInstructDataset(dataset['train'])
valid_data = MMInstructDataset(dataset['validation'])

In [ ]:
def collate_fn(batch):
  questions, answers, images = zip(*batch)
  inputs = processor(text=list(questions), images=list(images), return_tensors="pt", padding=True)
  return inputs, answers

In [ ]:
batch_size = 1
num_workers = 0

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=collate_fn)
valid_loader = DataLoader(dataset=valid_data, batch_size=batch_size, num_workers=num_workers, collate_fn=collate_fn)

In [ ]:
epochs = 2
num_training_steps = epochs * len(train_loader)
optimizer = AdamW(params=model.parameters(), lr=1e-5, optim_bits=8, is_paged=True)


In [ ]:
lr_sheduler = get_scheduler(name='linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
gradient_accumulation_steps = 8

In [ ]:
for epoch in range(epochs):
  model.train()
  train_loss = 0
  i = -1
  for inputs, answers in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
    i += 1
    input_ids = inputs['input_ids'].to('cuda')
    pixel_values = inputs['pixel_values'].to('cuda')
    labels = processor.tokenizer(text=answers, return_tensors='pt', padding=True, return_token_type_ids=False)['input_ids'].to('cuda')
    output = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
    loss = output.loss/gradient_accumulation_steps
    loss.backward()
    if i % gradient_accumulation_steps == 0:
      optimizer.step()
      lr_sheduler.step()
      optimizer.zero_grad()
      train_loss += loss.item()
    average_training_loss = train_loss / len(train_loader)
    print(f"Average training loss: {average_training_loss}")

    model.eval()
    valid_loss = 0
    with torch.no_grad():
      for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{epochs}"):
        inputs, answers = batch
        input_ids = inputs['input_ids'].to('cuda')
        pixel_values = inputs['pixel_values'].to('cuda')
        labels = processor.tokenizer(text=answers, padding=True, return_tensors='pt', return_token_type_ids=False)['input_ids'].to('cuda')
        output = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
        loss = output.loss
        valid_loss += loss.item()
      average_valid_loss = valid_loss / len(valid_loader)
      print(f"Average validation loss: {average_valid_loss}")

Training Epoch 1/2:   0%|          | 0/12726 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Average training loss: 3.95903830143822e-05



Validation Epoch 1/2:  36%|███▌      | 1524/4241 [15:45<28:24,  1.59it/s]